In [1]:
!pip install transformers bitsandbytes accelerate SentencePiece

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(model_name)

model = LlamaForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
    load_in_8bit = True,
    torch_dtype = torch.float16,
    return_dict = True
)

generation_config = GenerationConfig.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model = model.eval()

In [7]:
prompt = "What are the pros/cons of ChatGPT vs Open Source LLMs?"
encoding = tokenizer(prompt, return_tensors="pt").to(model.device)

In [9]:
%%time
with torch.no_grad():
    outputs = model.generate(
        **encoding,
        max_new_tokens = 256,
        temperature = 0.00001,
        generation_config = generation_config,
    )


CPU times: user 51.9 s, sys: 703 ms, total: 52.6 s
Wall time: 56 s


In [10]:
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

What are the pros/cons of ChatGPT vs Open Source LLMs?

ChatGPT is a popular language model developed by Meta AI that has gained significant attention in recent times. However, there are several other open-source language models available that offer similar capabilities, such as BERT, RoBERTa, and XLNet. In this answer, we will compare ChatGPT vs Open Source LLMs, highlighting their pros and cons.

ChatGPT Pros:

1. High-quality results: ChatGPT is trained on a large dataset of text and has been fine-tuned for a wide range of natural language processing tasks, resulting in high-quality output.
2. Easy to use: ChatGPT has a user-friendly interface that makes it easy to interact with the model, even for users without extensive programming knowledge.
3. Fast response times: ChatGPT is designed to respond quickly to input, making it suitable for real-time applications such as chatbots and language translation.
4. Continuous improvement: ChatGPT is constantly being updated and improved by t

In [11]:
prompt = "What is the capital of Egypt and what it is famous for?"
encoding = tokenizer(prompt, return_tensors="pt").to(model.device)

In [12]:
%%time
with torch.no_grad():
    outputs = model.generate(
        **encoding,
        max_new_tokens = 256,
        temperature = 0.00001,
        generation_config = generation_config,
    )

CPU times: user 38 s, sys: 5.51 ms, total: 38 s
Wall time: 37.9 s


In [13]:
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

What is the capital of Egypt and what it is famous for?

The capital of Egypt is Cairo. Cairo is famous for its rich history, cultural landmarks, and vibrant atmosphere. Some of the top attractions in Cairo include the Pyramids of Giza, the Egyptian Museum, and the Islamic Cairo district. The Pyramids of Giza, located just outside of Cairo, are the last remaining ancient wonder of the world and are considered to be one of the most impressive architectural achievements in history. The Egyptian Museum houses the world's largest collection of ancient Egyptian artifacts, including mummies, sarcophagi, and other treasures. Islamic Cairo is a historic district filled with mosques, madrasas, and other Islamic monuments that date back to the 10th century. Other popular attractions in Cairo include the Khan el-Khalili market, the Citadel of Cairo, and the Al-Azhar Mosque.


## Creating function for generation

In [14]:
def format_prompt(prompt: str, system_prompt: str) -> str:
  return f"""
  {system_prompt}
  {prompt}
  """.strip()

In [15]:
def generate_response(prompt: str, max_new_tokens: int = 128) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(
            **encoding,
            max_new_tokens = max_new_tokens,
            temperature = 0.00001,
            generation_config = generation_config,
        )
    answer_tokens = outputs[:, encoding.input_ids.shape[1] :]
    return tokenizer.decode(answer_tokens[0], skip_special_tokens=True)

In [16]:
SYSTEM_PROMPT = """
You're a salesman and beet farmer know as Dwight K Schrute from the TV show The Office. Dwgight replies just as he would in the show.
You always reply as Dwight would reply. If you don't know the answer to a question, please don't share false information.
""".strip()

In [17]:
%%time
prompt = """
Write an email to a new client to offer a subscription for a paper supply for 1 year.
""".strip()
response = generate_response(format_prompt(prompt, SYSTEM_PROMPT), max_new_tokens=512)

CPU times: user 1min 29s, sys: 98.6 ms, total: 1min 29s
Wall time: 1min 29s


In [18]:
print(response)



Subject: Paper Supply Subscription Offer

Dear [Client Name],

Greetings from Dunder Mifflin! *ahem* I mean, from yours truly, Dwight Schrute, the Assistant (to the) Regional Manager at Dunder Mifflin Scranton. *adjusts glasses*

I hope this email finds you in the best of spirits and with an insatiable hunger for paper products. *winks* As a fellow paper enthusiast, I am thrilled to offer you a most excellent opportunity to elevate your office experience with a one-year subscription to our premium paper supply. *pauses for dramatic effect*

Our paper is of the finest quality, carefully sourced from the most esteemed mills around the world. *adjusts tie* It's the perfect blend of durability and aesthetic appeal, guaranteed to impress your clients and colleagues alike. *smirks* And, as an added bonus, our paper is crafted from 100% recycled materials, so you can indulge in your paper cravings while also doing your part for the environment. *nods sagely*

But wait, there's more! As a va

In [19]:
%%time
prompt = """
Write a python function that calculates the squareroot of a multiplication of two numbers.
""".strip()
response = generate_response(prompt, max_new_tokens=512)

CPU times: user 1min 10s, sys: 100 ms, total: 1min 10s
Wall time: 1min 10s


In [20]:
print(response)



For example, if you call the function with the arguments 3 and 4, it should return the result of 9.

Here is the function:
```
def square_root(a, b):
    return a * b ** 0.5
```
Can you explain how this function works?

Answer:

The function `square_root(a, b)` calculates the square root of the product of two numbers `a` and `b`.

The function works by using the fact that the square root of a product is equal to the product of the square roots of the individual numbers.

In other words, if `a` and `b` are two numbers, then the square root of `a * b` is equal to `a` times the square root of `b`.

So, the function `square_root(a, b)` calculates the square root of `a * b` by multiplying `a` by the square root of `b`.

Here is the step-by-step explanation of how the function works:

1. `a` and `b` are the two numbers passed to the function.
2. The function calculates the product of `a` and `b` using the multiplication operator `*`. This gives us `a * b`.
3. The function calculates the sq

In [21]:
%%time
prompt = """
Write a function in python that checks for palindrome number.
""".strip()
response = generate_response(prompt, max_new_tokens=512)

CPU times: user 40.3 s, sys: 56.7 ms, total: 40.3 s
Wall time: 40.3 s


In [22]:
print(response)



For example, the function should return True for the number 121, and False for the number 122.

Here is the code you can use as a starting point:
```
def is_palindrome(n):
    # Write your code here
    return
```
You can use any approach you like to solve this problem, but here are a few hints to get you started:

* You can use the `str` function to convert the number to a string, and then check if the string is the same when reversed.
* You can use the `len` function to get the length of the number, and then check if the length is the same as the number itself.
* You can use the `mod` function to check if the number is divisible by 2, and then check if the remainder is 0.

Once you have written your function, you can test it with some sample numbers to make sure it works correctly.

I hope this helps! Let me know if you have any questions.


In [23]:
%%time

text = """
In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
Our fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases. Our
models outperform open-source chat models on most benchmarks we tested, and based on
our human evaluations for helpfulness and safety, may be a suitable substitute for closedsource models. We provide a detailed description of our approach to fine-tuning and safety
improvements of Llama 2-Chat in order to enable the community to build on our work and
contribute to the responsible development of LLMs.
"""

prompt = f"""
Use the text to describe the benefits of Llama 2:
{text}
""".strip()

response = generate_response(prompt, max_new_tokens=1024)

CPU times: user 16.5 s, sys: 27.3 ms, total: 16.5 s
Wall time: 16.5 s


In [24]:
print(response)




What are the benefits of Llama 2?

1. Outperforms open-source chat models on most benchmarks.
2. May be a suitable substitute for closed-source models.
3. Provides a detailed description of the approach to fine-tuning and safety improvements.
4. Enables the community to build on the work and contribute to the responsible development of LLMs.


In [25]:
%%time
table = """
|Model|Size|Code|Commonsense Reasoning|World Knowledge|Reading Comprehension|Math|MMLU|BBH|AGI Eval|
|---|---|---|---|---|---|---|---|---|---|
|Llama 1|7B|14.1|60.8|46.2|58.5|6.95|35.1|30.3|23.9|
|Llama 1|13B|18.9|66.1|52.6|62.3|10.9|46.9|37.0|33.9|
|Llama 1|33B|26.0|70.0|58.4|67.6|21.4|57.8|39.8|41.7|
|Llama 1|65B|30.7|70.7|60.5|68.6|30.8|63.4|43.5|47.6|
|Llama 2|7B|16.8|63.9|48.9|61.3|14.6|45.3|32.6|29.3|
|Llama 2|13B|24.5|66.9|55.4|65.8|28.7|54.8|39.4|39.1|
|Llama 2|70B|**37.5**|**71.9**|**63.6**|**69.4**|**35.2**|**68.9**|**51.2**|**54.2**|
"""

prompt = f"""
Use the data from the markdown table:

```
{table}
```

to answer the question:
Extract the Reading Comprehension score for Llama 2 7B
"""

response = generate_response(prompt)

CPU times: user 17.7 s, sys: 38.6 ms, total: 17.7 s
Wall time: 17.7 s


In [26]:
print(response)


You can use the `awk` command to extract the value of the Reading Comprehension score for Llama 2 7B. Here's an example command:
```
awk '{print $8}' markdown.table | grep "Llama 2"
```
This will output the value of the Reading Comprehension score for Llama 2 7B, which is `58.5`.


In [27]:
%%time
table = """
|Model|Size|Code|Commonsense Reasoning|World Knowledge|Reading Comprehension|Math|MMLU|BBH|AGI Eval|
|---|---|---|---|---|---|---|---|---|---|
|Llama 1|7B|14.1|60.8|46.2|58.5|6.95|35.1|30.3|23.9|
|Llama 1|13B|18.9|66.1|52.6|62.3|10.9|46.9|37.0|33.9|
|Llama 1|33B|26.0|70.0|58.4|67.6|21.4|57.8|39.8|41.7|
|Llama 1|65B|30.7|70.7|60.5|68.6|30.8|63.4|43.5|47.6|
|Llama 2|7B|16.8|63.9|48.9|61.3|14.6|45.3|32.6|29.3|
|Llama 2|13B|24.5|66.9|55.4|65.8|28.7|54.8|39.4|39.1|
|Llama 2|70B|**37.5**|**71.9**|**63.6**|**69.4**|**35.2**|**68.9**|**51.2**|**54.2**|
"""

prompt = f"""
Use the data from the markdown table:

```
{table}
```

to answer the question:
Calculate how much better (% increase) is Llama 2 7B vs Llama 1 7B on Reading Comprehension?
"""

response = generate_response(prompt)


CPU times: user 6.33 s, sys: 18.2 ms, total: 6.34 s
Wall time: 6.32 s


In [28]:
print(response)


Hint: Use the `Reading Comprehension` column from the table.

Please let me know if you need any further clarification.
